In [ ]:
import requests
import json
import numpy as np
import pandas as pd
import pytz
from datetime import datetime, timedelta

In [ ]:
#List of symbols
symb_list = ['NFLX','AAPL','GOOG','ROKU','FB','BABA','GOOGL','MSFT','NVDA','AMD','GM','WMT','TGT','JPM','ORCL',
             'LUV','AAL','DAL','SBUX','UAA','SPY','CMG','HD','INTC','RTN','CMI','SYMC','NKTR','EBAY','WYNN',
             'TWTR','SAGE','SQ','AXP','TSLA','ICPT']

In [ ]:
##header of requests (the same for all endpoints)
headers = {
           "Accept":"application/json",
           "Authorization":"Bearer <access_token_here>",
}

In [ ]:
# Quotes endopoint
params = (
        ('symbols', ','.join(symb_list)),
)
response = requests.get('https://sandbox.tradier.com/v1/markets/quotes', headers=headers, params=params)

In [ ]:
#Getting data into a Pandas dataframe
df = pd.DataFrame(response.json()['quotes']['quote'])

In [ ]:
#Dict with Stock symbol and price
symb_dict = {}
_ = df.apply(lambda x: symb_dict.update({x['symbol'] : x['last']}), axis=1)

#### Computing expirations dates (depends on parameter n_dates)

In [ ]:
n_dates = 5

In [ ]:
dayofweek = datetime.today().weekday()

In [ ]:
dates = []
delta_friday = 4 - dayofweek
if delta_friday < 0:
    delta_friday = delta_friday + 7
for i in range(n_dates):
    dates.append(datetime.today() + timedelta(delta_friday + 7*i)) 

In [ ]:
### How many "levels" to keep for both calls and puts
n_levels = 10

### Main loop that gets option chains and keep/compute relevant fields

In [ ]:
dataframes = []
#Call options api for all symbols and for all expiration dates
for symb in df.symbol.values:
    for exp_date in dates:    
        #Calling options chain endpoint
        params = (
                ('symbol', symb),
                ('expiration', str(exp_date)[:10]),
        )
        response = requests.get('https://sandbox.tradier.com/v1/markets/options/chains', 
                                headers=headers, params=params)
      
        print(response)
        
        #Getting the data as a Pandas dataframe
        df_opt = pd.DataFrame(response.json()['options']['option'])        
        df_opt.rename(columns={'last':'premium'}, inplace=True)        
        df_opt.dropna(subset=['premium'], inplace=True)
        
        #percent_diff is a measure of how far is the strike price moving from the stock price. It is used to compute
        #level_diff.
        df_opt['percent_diff'] = df_opt.strike.apply(lambda x: ((x - symb_dict[symb]) * 100) / symb_dict[symb]) 
        df_opt['stock_price'] = symb_dict[symb]
    
        #Working on puts.Keeping n_levels below stock price.
        df_put = df_opt.loc[df_opt.option_type == 'put']                                          
        df_put = df_put.loc[df_put.strike < symb_dict[symb]]
        df_put['level_diff'] = df_put.percent_diff.abs().argsort().argsort() + 1                                 
        df_put = df_put.loc[df_put.level_diff < n_levels]

        #Working on calls. Keeping n_levels above stock price.
        df_call = df_opt.loc[df_opt.option_type == 'call']
        df_call = df_call.loc[df_call.strike > symb_dict[symb]]
        df_call['level_diff'] = df_call.percent_diff.abs().argsort().argsort() + 1 
        df_call = df_call.loc[df_call.level_diff < n_levels]
        
        #Concatenating calls and puts
        df_res = pd.concat([df_put, df_call], ignore_index=True, axis=0)
        
        #Sorting dataframe and keeping relevant columns.
        df_res = df_res.loc[:, ['underlying', 'stock_price', 'option_type', 'expiration_date', 'premium', 'strike', 'percent_diff', 'level_diff', 'description']]
        df_res.sort_values(by='percent_diff', axis='index', inplace=True)
        df_res = df_res.reset_index(drop=True)
        df_res['factor'] = df_res.apply(lambda x: x['premium'] / x['stock_price'], axis=1)
        
        #all dataframes are added to 'dataframes' list
        dataframes.append(df_res)
  

In [ ]:
#Final datframe with all relevant information
df_factors = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_factors.shape

### Creating a bar plot from df_factors dataframe

In [ ]:
import holoviews as hv
from bokeh.server.server import Server
from holoviews import opts
hv.extension('bokeh')

In [ ]:
table = hv.Table(df_factors)

In [ ]:
plot = table.to.bars(kdims=['underlying'], 
                     vdims=['factor', 'stock_price', 'premium', 'strike', 'description'], 
                     groupby=['option_type', 'level_diff', 'expiration_date'])

In [ ]:
##Plotting inline in the notebook
plot.opts(opts.Bars(color=hv.Cycle('Category20'), show_legend=False, stacked=True, tools=['hover'], width=700))

### Deploying it as a Bokeh app

In [ ]:
renderer = hv.renderer('bokeh')
rend_app = plot.options(width=1500, height=800)
app = renderer.app(rend_app)

In [ ]:
server = Server({'/':app}, port=8001, allow_websocket_origin=["*"])
server.start()

In [ ]:
server.show('/')

In [ ]:
# server.stop()